<a href="https://colab.research.google.com/github/sonhs99/NLP/blob/main/implementation/BERT_KoChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT 모델 구현

사용 데이터: 인공적으로 제작된 문답 쌍 11,876개 (https://github.com/songys/Chatbot_data)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MODEL_PATH = 'drive/MyDrive/weights/BERT/'
PREV_MODEL_NAME = None #'BERTv1'
MODEL_NAME = 'BERTv3'

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import numpy as np
import urllib.request
import pandas as pd

urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


## 1. 데이터 정제 및 사전 구축

In [ ]:
import re

# 데이터 정제
# 문장부호를 단어로부터 분리

def preprocess(sentence):
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    return sentence

questions = []
for sentence in train_data['Q']:
    questions.append(preprocess(sentence))

answers = []
for sentence in train_data['A']:
    answers.append(preprocess(sentence))

print(questions[:5])
print(answers[:5])

train_Q, train_A = questions[:-1000], answers[:-1000]
test_Q, test_A = questions[-1000:], answers[-1000:]

['12시 땡 !', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']
['하루가 또 가네요 .', '위로해 드립니다 .', '여행은 언제나 좋죠 .', '여행은 언제나 좋죠 .', '눈살이 찌푸려지죠 .']


In [ ]:
import tensorflow_datasets as tfds

# Tokenizer 선언
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13
)

# 사전 크기 조정
# 문자열에 대한 사전 크기에 시작, 종결 토큰 추가
start_token, delimiter_token, mask_token = [tokenizer.vocab_size], [tokenizer.vocab_size + 1], [tokenizer.vocab_size + 2]
vocab_size = tokenizer.vocab_size + 3

print('시작 토큰 :', start_token)
print('종결 토큰 :', delimiter_token)
print('사전 크기 :', vocab_size)

시작 토큰 : [8178]
종결 토큰 : [8179]
사전 크기 : 8181


In [ ]:
print('원본 :', questions[20])
tokenized_string = tokenizer.encode(questions[20])
print('부호화 문자열 :', tokenized_string)
print('복호화 :', tokenizer.decode(tokenized_string))

원본 : 가스비 비싼데 감기 걸리겠어
부호화 문자열 : [5766, 611, 3509, 141, 685, 3747, 849]
복호화 : 가스비 비싼데 감기 걸리겠어


In [ ]:
max_length = 128

def tokenize_and_filter(inputs, outputs):
    tokenized_inputs, tokenized_outputs = [], []

    for s1, s2 in zip(inputs, outputs):
        s1 = tokenizer.encode(s1)
        s2 = tokenizer.encode(s2)

        tokenized_inputs.append(s1)
        tokenized_outputs.append(s2)

    return tokenized_inputs, tokenized_outputs

train_Q, train_A = tokenize_and_filter(train_Q, train_A)
# print(train_Q.shape, train_A.shape)

In [ ]:
batch_size = 128
buffer_size = 20000

def dataset_generator():
    length = len(train_Q)
    for idx in range(length):
        answer_rd = np.random.rand() > 0.5
        answer_idx = idx
        if answer_rd:
            while answer_idx == idx:
                answer_idx = np.random.randint(0, length)
        (input, segment_input), output = data_preprocess(train_Q[idx], train_A[answer_idx])
        yield (input, segment_input), (output, (int(not answer_rd),))

def data_preprocess(question, answer):
    q_len, a_len = len(question), len(answer)
    input = np.array(start_token + question + delimiter_token + answer + delimiter_token)
    output = input.copy()
    segment_input = [0] * (q_len + 2) + [1] * (a_len + 1)

    mask = np.random.choice([0, 1, 2, 3], q_len + a_len + 3, replace=True, p=[0.85, 0.12, 0.015, 0.015])
    random_array = np.random.randint(0, vocab_size, size=q_len + a_len + 3)
    # mask[0] = mask[q_len + 1] = mask[q_len + a_len + 2] = 0
    output = np.where(mask == 0, 0, output)
    input = np.where(mask == 1, mask_token[0], input)
    input = np.where(mask == 2, random_array, input)

    input = tf.keras.preprocessing.sequence.pad_sequences([input], maxlen=max_length, padding='post')
    segment_input = tf.keras.preprocessing.sequence.pad_sequences([segment_input], maxlen=max_length, padding='post')
    output = tf.keras.preprocessing.sequence.pad_sequences([output], maxlen=max_length, padding='post')
    return (input[0], segment_input[0]), output[0]

def create_dataset(questions, answers):
    dataset = tf.data.Dataset.from_generator(dataset_generator,
                ((tf.int32, tf.int32,),(tf.int32, tf.int32)),
                (((None,),(None,)),((None,), (1,)))
                )

    dataset = dataset.cache()
    dataset = dataset.shuffle(buffer_size)
    dataset = dataset.repeat()
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

train_dataset = create_dataset(train_Q, train_A)

## 2. Custom Layer 선언

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, dim):
        super(PositionalEncoding, self).__init__()
        self.sequence_length = sequence_length
        self.pos_vec = tf.keras.layers.Embedding(sequence_length, dim)

    def call(self, x):
        vec = tf.range(self.sequence_length)
        return x + self.pos_vec(vec)[:tf.shape(x)[1]]

In [ ]:
def attention(query, key, value, mask):
    depth = tf.cast(tf.shape(key)[-1], dtype=tf.float32)
    logit = tf.matmul(query, key, transpose_b=True) / tf.math.sqrt(depth)

    if mask is not None:
        logit += (mask * -1e9)

    dist = tf.nn.softmax(logit, axis=-1)
    return tf.matmul(dist, value)

class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, num_head, dim):
        super(AttentionLayer, self).__init__()

        assert dim % num_head == 0

        self.query = tf.keras.layers.Dense(dim, use_bias=False)
        self.key = tf.keras.layers.Dense(dim, use_bias=False)
        self.value = tf.keras.layers.Dense(dim, use_bias=False)

        self.w_o = tf.keras.layers.Dense(dim, use_bias=False)

        self.num_head = num_head
        self.dim = dim
        self.head_size = dim // num_head

    def split_value(self, x, batch_size):
        split = tf.reshape(x, [batch_size, -1, self.num_head, self.head_size])  # [batch, num_vocab, num_head, head_dim]
        return tf.transpose(split, [0, 2, 1, 3])                                # [batch, num_head, num_vocab, head_dim]

    def call(self, inputs):
        query = self.query(inputs[0])
        key = self.key(inputs[1])
        value = self.value(inputs[2])
        mask = inputs[3]

        batch_size = tf.shape(query)[0]

        query = self.split_value(query, batch_size)
        key = self.split_value(key, batch_size)
        value = self.split_value(value, batch_size)

        attention_value = attention(query, key, value, mask)                # [batch, num_head, num_vocab, head_dim]
        attention_value = tf.transpose(attention_value, perm=[0, 2, 1, 3])  # [batch, num_vocab, num_head, head_dim]
        attention_value = tf.reshape(attention_value, [batch_size, -1, self.dim]) # [batch, num_vocab, d_model] (num_head * head_dim = d_model)
        return self.w_o(attention_value)

In [ ]:
class EmbeddingTransposeLayer(tf.keras.layers.Layer):
    def __init__(self, embedding_layer: tf.keras.layers.Embedding, **kargs):
        super(EmbeddingTransposeLayer, self).__init__(**kargs)
        self.embeddings = embedding_layer

    def build(self, input_shape):
        self.custom_weights = self.embeddings.weights[0]
        self.build = True

    def compute_output_shape(self, input_shape):
        return input_shape[0], tf.keras.backend.int_shape(self.custom_weights)[0]

    def call(self, x):
        return tf.keras.backend.dot(
            x, tf.keras.backend.transpose(self.custom_weights))

## 3. Decoder 블록 및 GPT 모델 생성

In [ ]:
def encoder_block(dif, num_head, dim, dropout):
    input = tf.keras.layers.Input([None, dim])
    padding_input = tf.keras.layers.Input([1, 1, None])

    attention_x = AttentionLayer(num_head, dim)([input, input, input, padding_input])
    attention_x = tf.keras.layers.Dropout(dropout)(attention_x)
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(input + attention_x)

    ffnn_x = tf.keras.layers.Dense(dif, activation='relu', kernel_regularizer='l2')(x)
    ffnn_x = tf.keras.layers.Dense(dim, kernel_regularizer='l2')(ffnn_x)

    ffnn_x = tf.keras.layers.Dropout(dropout)(ffnn_x)
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x + ffnn_x)
    return tf.keras.Model(inputs=[input, padding_input], outputs=x)

def encoder(vocab_size, num_layers, dif, num_head, dim, dropout):
    input = tf.keras.layers.Input([None,])
    padding_input = tf.keras.layers.Input([1, 1, None])

    positional_encoding = PositionalEncoding(vocab_size, dim)(input)
    output = tf.keras.layers.Dropout(dropout)(positional_encoding)

    for i in range(num_layers):
        output = encoder_block(dif, num_head, dim, dropout)([output, padding_input])

    return tf.keras.Model(inputs=[input, padding_input], outputs=output, name='encoder')

In [ ]:
def padding_mask(x):
    mask = tf.cast(tf.math.equal(x, 0), dtype=tf.float32)
    return mask[:, tf.newaxis, tf.newaxis, :]

def bert_model(vocab_size, num_layers, dif, num_head, dim, dropout):
    input = tf.keras.layers.Input([None,], name='input')
    segment_input = tf.keras.layers.Input([None,], name='seg_input')

    embedding_layer = tf.keras.layers.Embedding(vocab_size, dim)
    embedding = embedding_layer(input)
    pad_mask = tf.keras.layers.Lambda(padding_mask, output_shape=[1, 1, None])(input)

    enc = encoder(vocab_size, num_layers, dif, num_head, dim, dropout)([embedding, pad_mask])

    # output = EmbeddingTransposeLayer(embedding_layer, name='output')(enc)
    output = tf.keras.layers.Dense(vocab_size, name='mask_output')(enc)
    sen_output = tf.keras.layers.Dense(2, name='sentense_output')(enc)
    return tf.keras.Model(inputs=[input, segment_input], outputs=[output, sen_output])

In [ ]:
dim = 768
num_layers = 12
num_head = 12
diff = 3072
dropout = 0.1

model = bert_model(
    vocab_size=vocab_size,
    num_layers=num_layers,
    dif=diff,
    dim=dim,
    num_head=num_head,
    dropout=dropout
)

if PREV_MODEL_NAME: model.load_weights(MODEL_PATH+PREV_MODEL_NAME+'.h5')

## 4. 모델 Compile 및 학습

In [ ]:
class TransformerSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, dim, warmup_step=4000):
        self.d_model = tf.cast(dim, tf.float32)
        self.warmup_step = warmup_step

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        temp1 = tf.math.rsqrt(step)
        temp2 = step * (self.warmup_step ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(temp1, temp2)

    def get_config(self):
        config = {
            'dim': self.d_model,
            'warmup_step': self.warmup_step,
        }
        return config

In [ ]:
# lr = tf.keras.optimizers.schedules.CosineDecay(
#     initial_learning_rate=0,
#     decay_steps=2000,
#     warmup_target=2.5e-4,
#     warmup_steps=2000
# )
opt = tf.optimizers.Adam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-9)

# def accuracy(y_true, y_pred):
#     y_true = tf.reshape(y_true, shape=(-1, max_length))
#     return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

# def perplexity(y_true, y_pred):
#     loss = loss_function(y_true, y_pred)
#     return tf.math.exp(loss)

def mask_loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, max_length))
    loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none'
    )(y_true, y_pred)
    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)
    return tf.reduce_mean(loss)

def seg_loss_function(y_true, y_pred):
    loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none'
    )(y_true, y_pred[0])
    return tf.reduce_mean(loss)

model.compile(
    optimizer=opt, loss=[mask_loss_function, seg_loss_function])

In [ ]:
model.summary(expand_nested=True)

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 768)            6283008   ['input[0][0]']               
                                                                                                  
 lambda (Lambda)             (None, 1, 1, None)           0         ['input[0][0]']               
                                                                                                  
 encoder (Functional)        (None, None, 768)            9130060   ['embedding[0][0]',           
                                                          8          'lambda[0][0]']       

In [ ]:
epoch = 400
history = model.fit(train_dataset, epochs=epoch, steps_per_epoch=len(train_Q)//batch_size, callbacks=[
    tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, start_from_epoch=100)
])

Epoch 1/400
84/84 [==============================] - 120s 891ms/step - loss: 257.1596 - mask_output_loss: 0.1278 - sentense_output_loss: 1.0489
Epoch 2/400
84/84 [==============================] - 75s 888ms/step - loss: 177.3203 - mask_output_loss: 0.1079 - sentense_output_loss: 0.7377
Epoch 3/400
84/84 [==============================] - 75s 889ms/step - loss: 117.7796 - mask_output_loss: 0.1074 - sentense_output_loss: 0.7290
Epoch 4/400
84/84 [==============================] - 75s 888ms/step - loss: 77.5280 - mask_output_loss: 0.1070 - sentense_output_loss: 0.7222
Epoch 5/400
84/84 [==============================] - 75s 888ms/step - loss: 51.3098 - mask_output_loss: 0.1070 - sentense_output_loss: 0.7248
Epoch 6/400
84/84 [==============================] - 75s 888ms/step - loss: 34.4887 - mask_output_loss: 0.1072 - sentense_output_loss: 0.7203
Epoch 7/400
84/84 [==============================] - 74s 887ms/step - loss: 23.7068 - mask_output_loss: 0.1070 - sentense_output_loss: 0.7141
Ep

KeyboardInterrupt: ignored

In [ ]:
import pandas as pd

model.save_weights(MODEL_PATH + MODEL_NAME+'.h5', save_format='h5')

hist = pd.DataFrame(history.history)
hist_filename = MODEL_NAME + '.json'
with open(MODEL_PATH + hist_filename, 'w') as file:
    hist.to_json(file)

NameError: ignored

## 5. Test

In [ ]:
idx = 800
print('Q :', questions[idx])
print('A :', answers[idx+1])
(input, seg_input), _ = data_preprocess(train_Q[idx], train_A[idx])
input = input[np.newaxis, :]
seg_input = seg_input[np.newaxis, :]
mask_output, sen_output = model.predict([input, seg_input])
mask_output = mask_output[0].argmax(axis=1)
sen_output = sen_output[0, 0].argmax()
recovered = tokenizer.decode(
        [x for x in mask_output if 0 < x < tokenizer.vocab_size]
    )
print(recovered)
print(sen_output)

Q : 내 얼굴이 읽히나
A : 어머어머 궁금하네요 .
1/1 [==============================] - 0s 167ms/step

1


In [ ]:
def eval(tokens):
    answer = tf.expand_dims(start_token + tokens, 0)
    for i in range(max_length):
        prediction = model(inputs=answer, training=False)
        prediction = prediction[:, -1:, :]
        next_token = tf.cast(tf.argmax(prediction, axis=-1), dtype=tf.int32)

        if tf.equal(next_token, end_token[0]):
            break

        answer = tf.concat([answer, next_token], axis=-1)
    return tf.squeeze(answer, axis=0)

def pred(sentence):
    sentence = preprocess(sentence)
    tokenized = tokenizer.encode(sentence)

    answer_token = eval(tokenized)
    answer = tokenizer.decode(
        [x for x in answer_token if x < tokenizer.vocab_size]
    )

    print(f"단어: {sentence}")
    print(f'생성(Token): {answer_token}')
    print(f'생성: {answer}')

    return answer

In [ ]:
pred('라면')

단어: 라면
생성(Token): [8178 4423 8157 8102 8059   31 7356 7954 2380   32    1]
생성: 라면봉지 레시피 그대로 해보세요 .


'라면봉지 레시피 그대로 해보세요 .'

In [ ]:
pred('밥')

단어: 밥
생성(Token): [8178 1330 4192  456  505    1]
생성: 밥심으로 사는 거죠 .


'밥심으로 사는 거죠 .'

In [ ]:
pred('땅')

단어: 땅
생성(Token): [8178 2662  657 1322]
생성: 땅이나 살까


'땅이나 살까'